In [1]:

import time
import random
import numpy as np
import math as m
from scipy import ndimage
import cv2
import itertools
import torch


from individual import Individual
from rotate import rotate

from __future__ import division
      

def close_cv2():
    for ind in range(10):
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        
def normalize(vec):
    vec_mag = np.linalg.norm(vec)
    vec_mag = np.maximum(vec_mag, EPSILON)
    vec_norm = vec / vec_mag
    return vec_norm

# differentiable equivalent of np.where
# cond could be a FloatTensor with zeros and ones
def where(cond, x_1, x_2):
    ones = cond.clone().zero_() + 1
    return (cond * x_1) + ((ones-cond) * x_2)
 
EPSILON = .0000001
    
class Simulation():
    
    def __init__(self, num_agents=400, number_of_states=2, arena_size=1000,
                 max_turning_rate=114.5, body_size=4.0, zoa=12.0,
                 zor=2.0, zoo=12.0, speed=10, angular_error_sd=0.0, 
                 ko=0.25, ka=0.25, timestep=0.2, periodic_boundary=True,
                 draw_sim=True, state_threshold=0.5, num_active=.5,
                 omega=.5, video=False, video_file='testvid.avi',
                 social_weight=0.85):
        
        self.num_agents = num_agents
        self.number_of_states = number_of_states
        self.arena_size = arena_size
        self.max_turning_rate = max_turning_rate
        self.body_size = body_size
        self.zoa = zoa * body_size
        self.zor = zor * body_size
        self.zoo = zoo * body_size
        self.speed = speed
        self.angular_error_sd =angular_error_sd
        self.ko = ko
        self.ka = ka
        self.timestep = timestep
        
        self.top_left = np.zeros(2)
        self.bottom_right = np.ones(2) * arena_size
        self.number_of_cells = int(arena_size // zoa)
        self.cell_size = arena_size / self.number_of_cells
        
        self.agents = self.generate_agents()
        
        self.periodic_boundary = periodic_boundary
        self.social_weight = social_weight
        self.draw_sim = draw_sim
        self.state_threshold = state_threshold
        self.num_active = int(num_agents * num_active)
        self.omega = omega
        self.video = video
        self.video_file = video_file
        
        #output of the video recorder, if enabled
        self.out = None 
        
    def initialize_video(self):
        fourcc = cv2.VideoWriter_fourcc(*'ffv1')
        out = cv2.VideoWriter(filename=self.video_file, fourcc=fourcc, fps=30, 
                              frameSize=(int(self.arena_size), int(self.arena_size)), isColor=True)
        return out

    
    def create_mask(self, r, inner_r, outer_r):
        """Return a mask that is true is distance is within the specified range

        arguments:
        r -- matrix containg distances between all individuals: shape() = (n,m)
        inner_r -- minimum distance (exclusive)
        outer_r -- maximum distance (exclusive)

        return matrix shape = (n,m,2)
        """
        mask = (r > inner_r) * (r < outer_r) * -1 + 1
        mask = torch.unsqueeze(mask, 2)
        mask = mask.repeat(1,1,2)
        return(mask)
    

    def create_mask_1d(self, r, inner_r, outer_r):
        """Return a mask that is true is distance is within the specified range

        arguments:
        r -- matrix containg distances between all individuals: shape() = (n,m)
        inner_r -- minimum distance (exclusive)
        outer_r -- maximum distance (exclusive)

        return matrix shape = (n,m,1)
        """
        mask = (r > inner_r) * (r < outer_r) * -1 + 1
        return(mask)

    def zone_vec_v(self, expanded_val, mask):
        """Apply mask to velocities calculate social alignment
        """
        vals = expanded_val.repeat(1, self.num_agents, 1)
        #take term from vals where mask is true and from zeros when false        
        vals = vals.masked_fill_(mask, 0.0)
        return(torch.sum(vals, 0))

    def zone_vec_x(self, rel_pos, mask):
        #take term from rel_pos where mask is true and from zeros when false
        vals = rel_pos.masked_fill_(mask, 0.0)
        return(torch.sum(vals, 0))
        
        
    def main_graph(self):
#         test = [cell for cell in self.cell_to_positions]
#         test_tensor = torch.FloatTensor(test[0])
#         #print('0: ', test[0], ' 1: ', test[1])
#         test_concat = torch.cat([test_tensor, torch.FloatTensor(test[1])])
#         #print(test_concat)
#         list_cells = []
#         for cell in self.cell_to_positions:
#             #print('cell: ', cell)
#             #print('type: ', type(cell))
#             #list_cells.append(torch.FloatTensor(cell))
        # list_cells = [torch.FloatTensor(cell) for cell in self.cell_to_positions] 
        self.x = torch.cat([torch.cuda.FloatTensor(cell) for cell in self.cell_to_positions])
        self.v = torch.cat([torch.cuda.FloatTensor(cell) for cell in self.cell_to_velocities])
        self.state = torch.cat([torch.cuda.FloatTensor(cell) for cell in self.cell_to_states])
        self.id = torch.cat([torch.cuda.IntTensor(cell) for cell in self.cell_to_agents])

        expanded_x1 = torch.unsqueeze(self.x, 0)  
        expanded_v1 = torch.unsqueeze(self.v, 0)
        expanded_state1 = torch.unsqueeze(self.state, 0)

        #periodic boundaries
        if self.periodic_boundary: 
            CellPos = [] #Will hold all the neighboring cells 
            for inx_1 in range(-1, 2): #Iterates over the neighboring cells
                for inx_2 in range(-1, 2):
                    # (0,0) position of each of the neighbouring cells (top left corner)
                    CellPos.append([inx_1*self.arena_size, inx_2*self.arena_size]) 
            CellPos = torch.cuda.FloatTensor(CellPos)
            AllParticles = torch.unsqueeze(CellPos, 1)  + expanded_x1  #Add particle position info to the neighbor cells
            AllParticles = AllParticles.view(-1, 2) # Reshape to single array of particles coordinates
            expanded_x2 = torch.unsqueeze(AllParticles, 1)
            AllParticlesV = expanded_v1.repeat(1, 9, 1)
            AllParticlesV = AllParticlesV.view(-1, 2)
            expanded_v2 = torch.unsqueeze(AllParticlesV, 1)
            AllParticlesState = expanded_state1.repeat(1, 9, 1)
            AllParticlesState = AllParticlesState.view(-1, 1)
            expanded_state2 = torch.unsqueeze(AllParticlesState, 1)

        else:
            expanded_x2 = torch.unsqueeze(self.x, 1)
            expanded_v2 = torch.unsqueeze(self.v, 1)
            expanded_state2 = torch.unsqueeze(self.state, 1)

        #distance based calculations

        #Distance between every pair of particles in x in every dimension (dx,dy)
        rx = expanded_x1 - expanded_x2 
        # square distane for each particle pair in each dimension  (dx^2,dx^2)
        rx2 = rx ** 2
        # absolute squar distance between every pair of particles(dx^2+dx^2)
        r2 = torch.sum(rx2, 2) 
        # absolute distance between every pair of particles
        r = torch.sqrt(r2)



        abs_r = torch.abs(r)
        # To avoid division by zero make min distance
        #larger then 0 this add to prevent simulation 
        #explosion if particles get too close
        r=torch.clamp(r, min=0.0002) 

        # radius of zone of alignment
        zor_r = torch.cuda.FloatTensor([2.0])
        # radius of zone of alignment
        zoo_r = torch.cuda.FloatTensor([12.0])
        # radius of zone of alignment
        zoa_r = torch.cuda.FloatTensor([12.0]) 

        
#         if self.periodic_boundary:
#             zeros = torch.FloatTensor([self.num_particles * 9, self.num_particles, 2]).zero_()

#         else:
#             zeros = torch.FloatTensor([self.num_particles, self.num_particles, 2]).zero_()
        
        r_expanded = torch.unsqueeze(r, 2)
        r_expanded = r_expanded.repeat(1, 1, 2)
        r_unit = rx / r_expanded    

        zor_mask = self.create_mask(abs_r, 0.0, zor_r)
        zor_x = self.zone_vec_x(r_unit, zor_mask)
        zor_x = zor_x * torch.cuda.FloatTensor([1.0])
        zor_sum = torch.sum(torch.abs(zor_x), 1)
        mask_zor_full = (zor_sum > torch.cuda.FloatTensor([0.0])).type(torch.cuda.FloatTensor)
        mask_zor_full = torch.unsqueeze(mask_zor_full,1)
        mask_zor_full = mask_zor_full.repeat(1, 2)

        zoo_mask = self.create_mask(abs_r, zor_r, zoo_r)
        zoo_v = self.zone_vec_v(expanded_v2, zoo_mask)
        zoo_v = zoo_v * torch.cuda.FloatTensor([1.0])  #3


        zoa_mask = self.create_mask(abs_r, zoo_r, zoa_r)
        zoa_x = self.zone_vec_x(r_unit, zoa_mask)
        zoa_x = zoa_x * torch.cuda.FloatTensor([-1.0])
                          

        state_mask = self.create_mask_1d(abs_r, zor_r, zoo_r)
        state_mask = torch.unsqueeze(state_mask, 2)

        zoa_zoo = torch.sum(zoa_x, 1) * torch.sum(zoo_v,1)
        mask_both_zones = zoa_zoo > torch.cuda.FloatTensor([0.0])


        #should be 0 if only one zone has individs, 1 if both
        both_zones = mask_both_zones.type(torch.cuda.FloatTensor)
        both_zones_expanded = torch.unsqueeze(both_zones, 1)
        both_zones_expanded = both_zones_expanded.repeat(1,2)
        
        point_five_vec = both_zones_expanded.clone().zero_()
        #.pow was cast to float before in tf
        outer_zones_v = torch.pow(point_five_vec, both_zones_expanded) * (zoa_x + zoo_v) 

        vnew = where(mask_zor_full, zor_x, outer_zones_v)
        vnew = torch.clamp(vnew, max=100000.0) #so speed can't blow up

        personal_weight = 1 - self.social_weight
        vnew = self.social_weight*vnew + personal_weight*self.v

        #vnew = ((1 - self.omega)*vnew + self.omega*self.personal_bias)

        vnew_mag = vnew ** 2
        vnew_mag = torch.sum(vnew_mag, 1) 
        vnew_mag = torch.sqrt(vnew_mag)
        vnew_mag = torch.clamp(vnew_mag, min=0.0002)
        vnew_mag = torch.unsqueeze(vnew_mag, 1)
        vnew_mag = vnew_mag.repeat(1, 2) 
        vnew = vnew / vnew_mag

        #turning_angle = self.vnew - self.v
        #turning_angle = torch.atan2(turning_angle[:, 1], turning_angle[:, 0]) 

        xnew = self.x + vnew*self.timestep

        #if repetitive boundary conditions are used make sure particle 
        #poistion dont exceed cell size
        if self.periodic_boundary:
            xnew=torch.fmod(xnew + self.arena_size, self.arena_size)
             
        #self.x = xnew
        #self.v = vnew

        #states

        
#         if self.periodic_boundary:
#             zeros_state = tf.zeros([self.num_particles * 9, self.num_particles, 1], tf.float32)

#         else:
#             zeros_state = tf.zeros([self.num_particles, self.num_particles, 1], tf.float32)
        
        
        state_active = self.zone_vec_v(expanded_state2, state_mask)
        ones_expanded_state2 = expanded_state2.clone().zero_() + 1
        state_ones = self.zone_vec_v(ones_expanded_state2, state_mask)
        
        state_active = torch.sum(state_active, 1)
        state_ones = torch.sum(state_ones, 1)
        state_fraction = state_active / state_ones

        ones_state = self.state.clone().zero_() + 1
        zeros_state = self.state.clone().zero_()
        mask_states = state_fraction < self.state_threshold
        self.state.masked_fill_(mask_states, 0.0)
        mask_states = state_fraction > self.state_threshold
        self.state.masked_fill_(mask_states, 1.0)
        
        for id in self.id:
            self.agents[id].r_center = xnew[id].cpu().numpy().astype(dtype='float64')
            self.agents[id].direction = vnew[id].cpu().numpy().astype(dtype='float64')
            self.agents[id].state = self.state[id]





        #noise = tf.random_normal(vnew.get_shape(), mean=0.0, stddev=self.noise_stddiv, dtype=tf.float32)
        #vnew = vnew + noise

            


    #@property
    def run_sim(self):
        WITH_TIMER = False
        arena_size_int = int(self.arena_size)
        
        if self.video:
            self.out = self.initialize_video()

        for i in range(100000):   
            [self.positions, self.velocities, self.init_state] = (
                session.run([self.xnew,self.vnew,self.state_new],feed_dict = {
                    self.x: self.positions, 
                    self.v: self.velocities,
                    self.state: self.init_state}))
        #----------------------Plot particles position real time---------------------------------------------       
            if (i%2==0) and (self.draw_sim==True or self.video==True):
                self.draw(arena_size_int)
                if cv2.waitKey(1) & 0xFF == 27:
                    break  #time delay
            sum_state = np.sum(self.init_state, 0) / self.num_agents
            if sum_state == 0 or sum_state == 1:
                break
        if self.video:
            self.out.release()
        session.close() 
        cv2.destroyAllWindows() 
        return sum_state[0]
    
    
    def draw(self, window_size):

        visualization = (np.ones((window_size, window_size, 3), np.uint8) * kc.blueberry_slate('np.uint8'))

        Cord=(np.array(self.positions)).astype(int) # Tranfer particles cordinates to numpy array format to use in plotting functin

        for i in range(len(Cord[:,0])):
            
            body_scale = .75
            
            if self.init_state[i] == 0:
                color = kc.red()
                
            else:
                color = kc.ivory()
            
            
            cv2.line(visualization, tuple(Cord[i,:]), 
                         tuple((Cord[i,:] - self.velocities[i,:] 
                              * 3 * self.body_size *body_scale).astype(int)),
                         color, 1, cv2.LINE_AA)
            
            cv2.circle(visualization,tuple(Cord[i,:]), 
                           int(self.body_size *body_scale), color,-1)

            

        visualization = cv2.cvtColor(visualization, cv2.COLOR_BGR2RGB)

        if self.draw_sim:    
            cv2.imshow("visualization", visualization)
            cv2.waitKey(1)
        if self.video:
            visualization = (visualization).astype('u1')
            self.out.write(visualization)

        
        
    def generate_agents(self):

        agents = []
        
        for indx_agents in range(self.num_agents):
            # needs to be a unit vector
            set_direction = np.array([1.0, 0.0])    
            set_direction = rotate(set_direction, random.random()*360.0)
            set_r_center = self.random_bounded_point()
            state = np.random.choice(np.arange(self.number_of_states)).astype(np.float)
            
            agents.append(
                Individual(set_r_center, set_direction, self.max_turning_rate,
                           self.speed, self.zoa, self.zoo, self.zor, self.angular_error_sd, 
                           self.body_size, state, self.ko, self.ka))
        return agents

    def _old_move_agents(self):
        #reset each individuals z buffer, sight_vec, and zoa_count
        for i in range(len(agents)):
            self.agents[i].zoa_count = 0
            self.agents[i].zor_count = 0
            self.agents[i].total_zoa = np.zeros(2)
            self.agents[i].total_zoo = np.zeros(2)
            self.agents[i].total_zor = np.zeros(2)
            self.agents[i].state_count = np.zeros(number_of_states)

        self.linked_list()
        self.implement_social_forces()


        for i in range(len(self.agents)):
            self.agents[i].move(self.arena_size, self.timestep)
            
    #def move_agents(self):
    
    def random_bounded_point(self):

        # Create randomly distributed co-ordinate in the simulated world
        range_x = self.bottom_right[0] - self.top_left[0]
        range_y = self.bottom_right[1] - self.top_left[1]

        random_x = random.random()
        random_y = random.random()

        random_x *= range_x
        random_y *= range_y
        random_point = np.array([random_x, random_y])

        return random_point
        


    def segment_individuals(self):

        # this implementation assumes equal x and y 

        total_cells = self.number_of_cells * self.number_of_cells
        
        self.cell_to_agents = [[] for x in range(total_cells)]
        self.cell_to_positions = [[] for x in range(total_cells)]
        self.cell_to_velocities = [[] for x in range(total_cells)]
        self.cell_to_states = [[] for x in range(total_cells)]
        cell_x = 0
        cell_y = 0

        for agent_id in range(len(self.agents)):
            # cell_x and cell_y determine which cell agents is in
            cell_x = self.agents[agent_id].r_center[0] // self.cell_size
            cell_y = self.agents[agent_id].r_center[1] // self.cell_size

            #calculate scalar cell id
            cell = int(cell_x * self.number_of_cells + cell_y)                      
            self.cell_to_agents[cell].append(agent_id) 
            self.cell_to_positions[cell].append(self.agents[agent_id].r_center)
            self.cell_to_velocities[cell].append(self.agents[agent_id].direction)
            self.cell_to_states[cell].append(self.agents[agent_id].state)
            
    def graphics(self):

        body_scale = .75


        colors = [[244,66,66],[244,158,66],[244,232,66],[185,244,66],[125,244,66],
                  [66,244,200],[66,217,244],[66,101,244],[176,66,244],[244,66,170],
                  [255,135,71],[255,255,255]]

        visualisation = np.zeros((self.arena_size, self.arena_size, 3), np.uint8)

        for i in range(len(self.agents)):

            color = colors[int(self.agents[i].state)]


            cv2.circle(visualisation, (int(self.agents[i].r_center[0]), 
                int(self.agents[i].r_center[1])), int(self.agents[i].body_size * body_scale), 
                color, -1, cv2.LINE_AA)


            cv2.line(visualisation, (int(self.agents[i].r_center[0]), 
                     int(self.agents[i].r_center[1])), 
                     (int(self.agents[i].r_center[0] - self.agents[i].direction[0] * 8.0),
                      int(self.agents[i].r_center[1] - self.agents[i].direction[1] * 8.0)), 
                     color, 1, cv2.LINE_AA)


        cv2.imshow("decision_making", visualisation)
        return (cv2.waitKey(1) & 0xFF == 27)



#     def compute_social_forces(i, j, agents, arena_size): 

#         temp_vector = np.zeros(2);

#         temp_vector = agents[j].r_center - agents[i].r_center

#         if abs(temp_vector[0]) > arena_size / 2:

#             if agents[i].r_center[0] < agents[j].r_center[0]:
#                 temp_vector[0] = (agents[j].r_center[0] 
#                                  - (agents[i].r_center[0] + arena_size))
#             else:
#                 temp_vector[0] = (agents[j].r_center[0]
#                                  - (agents[i].r_center[0] - arena_size))

#         if abs(temp_vector[1]) > arena_size / 2:

#             if (agents[i].r_center[1] < agents[j].r_center[1]):
#                 temp_vector[1] = (agents[j].r_center[1] 
#                                  - (agents[i].r_center[1] + arena_size))
#             else:
#                 temp_vector[1] = (agents[j].r_center[1] 
#                                  - (agents[i].r_center[1] - arena_size))

#         dist = np.linalg.norm(temp_vector)
#         dist = np.maximum(dist, EPSILON)

#         temp_vector_norm = temp_vector / dist

#         if dist < agents[i].zone_of_repulsion:
#             agents[i].total_zor += (-temp_vector_norm)
#             agents[i].zor_count += 1
#             agents[i].state_count[agents[j].state] += 1


#         elif dist < agents[i].zone_of_attraction: 
#             if dist < agents[i].zone_of_orientation:
#                 agents[i].total_zoo += agents[j].direction
#             agents[i].total_zoa += temp_vector
#             agents[i].zoa_count += 1
#             agents[i].state_count[agents[j].state] += 1







#     def implement_social_forces(agents):

#         #now have total_zod, total_zoo and total_zoa calculated for all individuals
#         for agent_indx in range(total_agents):
#             if agents[agent_indx].zor_count > 0:
#                 if (np.absolute(agents[agent_indx].total_zor[0]) < EPSILON and 
#                     np.absolute(agents[agent_indx].total_zor[1]) < EPSILON):
#                         agents[agent_indx].desired_direction = agents[agent_indx].direction 
#                 else:
#                     agents[agent_indx].desired_direction = normalize(agents[agent_indx].total_zor)


#             elif agents[agent_indx].zoa_count > 0:
#                 agents[agent_indx].desired_direction = (
#                     normalize(agents[agent_indx].total_zoo) * agents[agent_indx].ko +
#                     normalize(agents[agent_indx].total_zoa) * agents[agent_indx].ka + 
#                     agents[agent_indx].direction * (1 - agents[agent_indx].ko - agents[agent_indx].ka))

#                 if (np.absolute(agents[agent_indx].desired_direction[0]) < EPSILON and 
#                     np.absolute(agents[agent_indx].desired_direction[1]) < EPSILON):

#                     agents[agent_indx].desired_direction = agents[agent_indx].direction

#                 else:

#                     agents[agent_indx].desired_direction = normalize(agents[agent_indx].desired_direction)

#             if agents[agent_indx].zor_count > 0 or agents[agent_indx].zoa_count > 0:
#                 agents[agent_indx].state = np.argmax(agents[agent_indx].state_count)












In [5]:
#test area

random.seed()

total_frames = 10000

simulation = Simulation(num_agents=200, arena_size=200)



#Simulation starts HERE
for frame in range(total_frames):
    simulation.segment_individuals()
    simulation.main_graph()

    if frame % 1 == 0:
            kill = simulation.graphics()

    if kill:
        close_cv2()
        break




In [3]:
simulation.segment_individuals()
len(simulation.cell_to_positions)

6889

In [2]:
EPSILON = .0000001
total_agents = 400
TIMESTEP = 0.2
number_of_states = 10  

#Simulation start time
t1 = time.clock()
#Random generator engine from a time-based seed
random.seed()

#Set parameters

arena_size = 1000
top_left = np.array([0.0,0.0])
bottom_right = np.array([arena_size,arena_size])


sight_num = 166  #make even


max_turning_rate = 114.5
body_size = 4.0
zoa = 12.0 * body_size
zor = 2.0 * body_size
zoo = 12.0 * body_size 
speed = 10
color = [255,135,71]

angular_error_sd = 0.0

ko = 0.25
ka = 0.25


agents = []


#Simulation starts HERE

#Set up agents
for indx_agents in range(total_agents):
    # needs to be a unit vector
    set_direction = np.array([1.0, 0.0])    
    set_direction = rotate(set_direction, random.random()*360.0)
    set_r_center = random_bounded_point(bottom_right, top_left)
    state = np.random.choice(np.arange(number_of_states))


    agents.append(Individual(set_r_center, set_direction, max_turning_rate,
                             speed, zoa, zoo, zor, angular_error_sd, 
                             body_size, state, ko, ka))

for indx_j in itertools.count():

    move_agents(arena_size, zoa, agents)


    #draw to screen
    if indx_j % 1 == 0:
        kill = graphics(arena_size, agents, zoa)

    if kill:
        close_cv2()
        break

In [4]:
test = [[] for _ in range(10)]
for indx in range(10):
    for indy in range(indx):
        test[indx].append([indx,indy])
        
    

In [6]:
print(test)

[[], [[1, 0]], [[2, 0], [2, 1]], [[3, 0], [3, 1], [3, 2]], [[4, 0], [4, 1], [4, 2], [4, 3]], [[5, 0], [5, 1], [5, 2], [5, 3], [5, 4]], [[6, 0], [6, 1], [6, 2], [6, 3], [6, 4], [6, 5]], [[7, 0], [7, 1], [7, 2], [7, 3], [7, 4], [7, 5], [7, 6]], [[8, 0], [8, 1], [8, 2], [8, 3], [8, 4], [8, 5], [8, 6], [8, 7]], [[9, 0], [9, 1], [9, 2], [9, 3], [9, 4], [9, 5], [9, 6], [9, 7], [9, 8]]]


In [8]:
import torch
test_array = torch.cat((torch.FloatTensor(t) for t in test))
print(test_array)


    1     0
    2     0
    2     1
    3     0
    3     1
    3     2
    4     0
    4     1
    4     2
    4     3
    5     0
    5     1
    5     2
    5     3
    5     4
    6     0
    6     1
    6     2
    6     3
    6     4
    6     5
    7     0
    7     1
    7     2
    7     3
    7     4
    7     5
    7     6
    8     0
    8     1
    8     2
    8     3
    8     4
    8     5
    8     6
    8     7
    9     0
    9     1
    9     2
    9     3
    9     4
    9     5
    9     6
    9     7
    9     8
[torch.FloatTensor of size 45x2]



In [36]:
import torch

original_list_1 = []
for i in range(10):
    original_list_1.append(np.array([i,i/2]))
print(type(original_list_1[0]))
original_list_2 = []
for i in range(10):
    original_list_2.append(np.array([i/5,i/3]))
print(original_list_2)
tensor = torch.cat([torch.FloatTensor(original_list_1), torch.FloatTensor(original_list_2)])
print(tensor)

second_list_1 = []
for i in range(10):
    second_list_1.append(tensor[i].numpy())
    
second_list_2 = []
for i in range(10):
    second_list_2.append(tensor[i+10].numpy().astype(dtype='float64'))
    
    
print(second_list_2)

<class 'numpy.ndarray'>
[array([ 0.,  0.]), array([ 0.2       ,  0.33333333]), array([ 0.4       ,  0.66666667]), array([ 0.6,  1. ]), array([ 0.8       ,  1.33333333]), array([ 1.        ,  1.66666667]), array([ 1.2,  2. ]), array([ 1.4       ,  2.33333333]), array([ 1.6       ,  2.66666667]), array([ 1.8,  3. ])]

 0.0000  0.0000
 1.0000  0.5000
 2.0000  1.0000
 3.0000  1.5000
 4.0000  2.0000
 5.0000  2.5000
 6.0000  3.0000
 7.0000  3.5000
 8.0000  4.0000
 9.0000  4.5000
 0.0000  0.0000
 0.2000  0.3333
 0.4000  0.6667
 0.6000  1.0000
 0.8000  1.3333
 1.0000  1.6667
 1.2000  2.0000
 1.4000  2.3333
 1.6000  2.6667
 1.8000  3.0000
[torch.FloatTensor of size 20x2]

[array([ 0.,  0.]), array([ 0.2       ,  0.33333334]), array([ 0.40000001,  0.66666669]), array([ 0.60000002,  1.        ]), array([ 0.80000001,  1.33333337]), array([ 1.        ,  1.66666663]), array([ 1.20000005,  2.        ]), array([ 1.39999998,  2.33333325]), array([ 1.60000002,  2.66666675]), array([ 1.79999995,  3.     

In [ ]:
    # Part 2: Compute interactions
    for cell_x  in range(number_of_cells):
        for cell_y in range(number_of_cells):
           
            cell_id = cell_x * number_of_cells + cell_y
            if cell_to_agents_list[cell_id] != -1:

                for neighbor_x in range(cell_x - 1, cell_x + 2):
                    for neighbor_y in range(cell_y - 1, cell_y + 2):
                    
                        periodic_adjusted_x = neighbor_x;
                        periodic_adjusted_y = neighbor_y;
                        
                        if neighbor_x < 0:
                            periodic_adjusted_x += number_of_cells 
                        elif neighbor_x >= number_of_cells:
                            periodic_adjusted_x -= number_of_cells
                        
                        if neighbor_y < 0:
                            periodic_adjusted_y += number_of_cells
                        elif neighbor_y >=number_of_cells:
                            periodic_adjusted_y -= number_of_cells
                        
                        neighbor_cell_id = periodic_adjusted_x * number_of_cells + periodic_adjusted_y
                        
                        focal_agent_id = cell_to_agents_list[cell_id]
                        
                        while focal_agent_id != -1:
                            neighbor_agent_id = cell_to_agents_list[neighbor_cell_id]
                            
                            while neighbor_agent_id != -1:
                                if focal_agent_id != neighbor_agent_id:
                                    compute_social_forces(
                                        int(focal_agent_id), int(neighbor_agent_id), agents, arena_size)     
                                neighbor_agent_id = agents_to_agents_list[neighbor_agent_id]
                            focal_agent_id = agents_to_agents_list[focal_agent_id];